### New metric developed by Alessandro

In [1]:
import ee
ee.Initialize()

In [2]:
import geemap
Map = geemap.Map()

In [6]:
# ---- load datasets
# load MODIS data (daily 500m resolution)
collection = ee.ImageCollection('MODIS/006/MOD09GA').filterDate('2001-01-01', '2019-12-31')
# mask of pixels that were unchanged (until 2015)
MakMarco = ee.Image("users/marcogirardello/phenoutils/mask_unchanged_500m")
# grid for export to assets
worldgrid = ee.FeatureCollection('users/marcogirardello/phenoutils/grid_export_phenology')
# load large export grid
#exportgrid = ee.FeatureCollection('users/marcogirardello/phenoutils/large_grid')

In [7]:
# set dates
start_date = ee.Date.fromYMD(2001, 1, 1)
end_date   = ee.Date.fromYMD(2019, 12, 31)

In [4]:
#----- Required functions
# add NDVI to data
def addNDVI(image):
    image = image.updateMask(MakMarco.eq(1))
    return image.addBands(image.normalizedDifference(['sur_refl_b02','sur_refl_b01']).rename('NDVI')).float()

# function for extracting quality bits
def getQABits(image, start, end, mascara):
    # Compute the bits we need to extract.
    pattern = 0
    for i in range(start,end+1):
        pattern += 2**i
    # Return a single band image of the extracted QA bits, giving the     band a new name.
    return image.select([0], [mascara]).bitwiseAnd(pattern).rightShift(start)

# mask out low quality pixels (based on flags)
def maskPixels(image0):
    #Select the QA band
    QA = image0.select('state_1km')
    # Get the land_water_flag bits
    landWaterFlag = getQABits(QA, 3, 5, 'land_water_flag')
    #Get the cloud_state bits and find cloudy areas.
    cloud = getQABits(QA, 0, 1, 'cloud_state').expression("b(0) == 1 || b(0) == 2")
    # Get the cloud_shadow bit
    cloudShadows = getQABits(QA, 2, 2, 'cloud_shadow')
    # Get the Pixel is adjacent to cloud bit
    cloudAdjacent = getQABits(QA, 13, 13, 'cloud_adj')
    # Get the internal cloud flag
    cloud2 = getQABits(QA, 10, 10, 'cloud_internal')
    # Get the internal fire flag
    fire = getQABits(QA, 11, 11, 'fire_internal')
    # Get the MOD35 snow/ice flag
    snow1 = getQABits(QA, 12, 12, 'snow_MOD35')
    # Get the internal snow flag
    snow2 = getQABits(QA, 15, 15, 'snow_internal')
    # create mask
    mask = landWaterFlag.eq(1).And(cloud.Not()).And(cloudShadows.Not()).And(cloudAdjacent.Not()).And(cloud2.Not()).And(fire.Not()).And(snow1.Not()).And(snow2.Not())
    return image0.updateMask(mask) 
            

# utility function for temporal smoothing
def smooth_func(image):
    collection = ee.ImageCollection.fromImages(image.get('images'))
    return ee.Image(image).addBands(collection.mean().rename('mean'))

# masked smoothed dataset
def mask_smt(image):
    img = image.select('NDVI')
    img.updateMask(img.gt(0))
    return image.addBands(img)

# masked smoothed dataset
def unmask_img(image):
    return image.unmask()

# function for accumulating NDVI
def accumulate(image,list):
    # Get the latest cumulative NDVI of the list with
    # get(-1).  Since the type of the list argument to the function is unknown,
    # it needs to be cast to a List.  Since the return type of get() is unknown,
    # cast it to Image.
    previous = ee.Image(ee.List(list).get(-1)).toFloat()
    # Add the current anomaly to make a new cumulative NDVI image and Propagate metadata to the new image.
    added = image.toFloat().add(previous).toFloat().set('system:time_start', image.get('system:time_start'))
    return ee.List(list).add(added)


# add day of the year
def addDOY(image):
    return image.addBands(ee.Image.constant(ee.Number.parse(image.date().format('D'))).rename('DOY').float())

# calculate 50th percentile
def perc_50(img):
    # subtract the 50th percentile from each image, square to remove negative, append summed difference to image 
    dif = ee.Image(img).select(['mean']).subtract(Sum_NDVI.divide(2)).pow(ee.Image.constant(2)).multiply(-1).rename('quality')
    return img.addBands(dif)

# calculate 25th percentile
def perc_25(img):
    dif = ee.Image(img).select(['mean']).subtract(Sum_NDVI.divide(4)).pow(ee.Image.constant(2)).multiply(-1).rename('quality')
    return img.addBands(dif)

# calculate 75th percentile
def perc_75(img):
    dif = ee.Image(img).select(['mean']).subtract(Sum_NDVI.divide(4).multiply(3)).pow(ee.Image.constant(2)).multiply(-1).rename('quality')
    return img.addBands(dif)

# rename band
def rename_d(image):
    return Final_output_.select([b]).rename('value')

# smoothed mask for NDVI max
def mask_maxndvi(image):
    image = image.updateMask(count_valid.gte(30))
    return image.updateMask(image.select('NDVI').gt(0))

# rename ndvi for max ndvi dataset
def rename_maxndvi(b):
    return Final_output_.select([b]).rename('value')

# filter pixels greater than 10 %
def ndvi_filter10(image):
    return image.updateMask(image.gte(MaxNDVI_10))

# cumulative normalized
def cum_dividelast(image):
    return image.divide(last)

# filter smoothed map
def filt_smoothed(image):
    return image.updateMask(count_valid.gte(70))

# calculate deviations
def deviations_calc(image):
    tmp = image.select('mean').reproject(crs = 'SR-ORG:6974',scale = 463.3127165275).reduceNeighborhood(reducer='stdDev',kernel= ee.Kernel.square(6, 'pixels'),skipMasked =True)
    return image

In [9]:
# mask pixels on the basis of quality flags
MOD09masked = collection.filterDate(start_date, end_date).map(maskPixels)
# add NDVI band
MOD09ndvi = MOD09masked.map(addNDVI).select('NDVI')

In [73]:
# Loop through years
#years = list(range(2001, 2019+1))
years = list([2001,2019])

# polygons for export
polygons = list(range(2, 12+1))

for year in years:
    #print(year)
    # start and end date for a given year
    start_date = ee.Date.fromYMD(year,1,1)
    end_date = ee.Date.fromYMD(year,12,31)
    # filter dataset for a given year
    MOD09ndviY = MOD09ndvi.filterDate(start_date, end_date)
    # find max and divide by 10
    MaxNDVI_10 = MOD09ndviY.max().divide(10)
    # filter according to 10% rule
    MOD09ndviY = MOD09ndviY.map(ndvi_filter10)
    bw = 35
    # This field contains UNIX time in milliseconds
    timeField = 'system:time_start'
    # sort collection by date
    filteredMODIS = MOD09ndviY.sort('system:time_start')
    # Smoothing
    join = ee.Join.saveAll(matchesKey = 'images')
    diffFilter = ee.Filter.maxDifference(difference =1000 * 60 * 60 * 24 * bw, 
                                         leftField = timeField,rightField = timeField)
    threeNeighborJoin = join.apply(primary = filteredMODIS,secondary = filteredMODIS,condition = diffFilter)
    # get smoothed collection
    smoothed = ee.ImageCollection(threeNeighborJoin.map(smooth_func))
    # mask smoothed dataset
    smoothed_masked = smoothed.map(mask_smt)
    # count valid images
    count_valid = smoothed_masked.select('NDVI').count()
    # unmask smoothed dataset
    smoothed = smoothed.map(filt_smoothed)
    # select mean band
    smoothed = smoothed.select('mean')
    # Get the timestamp from the most recent image in the reference collection.
    time0 = smoothed.first().get('system:time_start')
    # Use imageCollection.iterate() to make a collection of cumulative NDVI over time.
    # Rename the first band 'NDVI'.
    first = ee.List([ee.Image(0).set('system:time_start', time0).select([0], ['mean']).toFloat()])
    # cumulate NDVI
    cumulative = ee.ImageCollection(ee.List(smoothed.iterate(accumulate, first)))
    # normalise
    last = cumulative.sort('system:time_start', False).first()
    # cumulative map normalised
    cumulativeNorm = cumulative.map(cum_dividelast)
    # deviations (these are described in the document sent by Alessandro)
    cumulativeStd10 = cumulativeNorm.map(deviations_calc)
    # calculate the mean of the deviations
    cumulativeStd10_mean = cumulativeStd10.mean().reproject(crs = 'SR-ORG:6974', scale = 463.3127165275)
    for polygon in polygons:
        # subset one broad area
        barea = exportgrid.filterMetadata('polyID','equals',polygon).geometry()
        # assetname
        assetname = 'deviations_'+str(year)+'_'+str(polygon)
        # export tile
        task = ee.batch.Export.image.toAsset(image=cumulativeStd10_mean,description=assetname,assetId = 'users/marcogirardello/phenodeviationsmetric/'+assetname,
                                     scale=463.3127165275,crs = 'EPSG:4326',maxPixels = 1e13,region = barea)
        task.start()    